In [3]:
# stopwords?
# stemmer or lemmatizer?
# count instead of tfidf?
# svd or highest feature frequency?
# removed all punctuation here
# 加入multinomialNB
# 维持pos:neg的比例
# pos tagging,换tagger
# scp -P 49233 kaijiang@master1.internal.datalaus.net:~/output_data.csv .

%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sparse

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from nltk.corpus import stopwords
from sklearn.cross_validation import StratifiedKFold
from sklearn.naive_bayes import BernoulliNB
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import recall_score

In [4]:

def read_json_self(path):
    g = open(path,'rb')
    for l in g:
        yield eval(l)
        
def getDF(path):
    i = 0
    df = {}
    for d in read_json_self(path):
        df[i] = d
        i+=1
    return pd.DataFrame.from_dict(df,orient='index')

kindle_data = getDF("reviews_Kindle_Store.json")


In [2]:
'''
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

kindle_data = getDF('reviews_Kindle_Store.json.gz')
'''

In [5]:
scores = kindle_data.overall
kindle_data.overall = kindle_data.overall.apply(lambda x : 'pos' if x > 3 else 'neg')

In [6]:
kindle_data.overall.value_counts()

pos    2582347
neg     623120
Name: overall, dtype: int64

In [7]:
def splitPosNeg(data):
    neg = kindle_data.loc[data.overall=='neg']
    pos = kindle_data.loc[data.overall=='pos']
    return [pos,neg]

[pos,neg] = splitPosNeg(kindle_data)

In [14]:
#preprocessing steps





#stemmer = PorterStemmer()
lemmatizer = nltk.WordNetLemmatizer()
stop = stopwords.words('english')
translation = string.maketrans(string.punctuation,' '*len(string.punctuation))

def preprocessing(line):
    tokens=[]
    line = line.translate(translation)
    line = nltk.word_tokenize(line.lower())
    for t in line:
        if(t not in stop):
            #stemmed = stemmer.stem(t)
            stemmed = lemmatizer.lemmatize(t)
            tokens.append(stemmed)
    return ' '.join(tokens)

In [47]:
pos_sample = pos.sample(frac=0.02)
neg_sample = neg.sample(frac=0.004)

# here i sample pos, so data more balanced and data size reduced
# so we get 5*6000:6000 reviews

In [48]:
pos_data = []
neg_data = []
#for p in pos.reviewText:
for p in pos_sample['reviewText']:
    pos_data.append(preprocessing(p))
    
#for p in neg.reviewText:
for n in neg_sample['reviewText']:
    neg_data.append(preprocessing(n))  

In [49]:
data = pos_data + neg_data
# remember this is sampled
labels = np.concatenate((pos_sample['overall'].values,neg_sample['overall'].values))
# labels = np.concatenate((pos['overall'].values,neg['overall'].values))

In [50]:
output_data = pd.DataFrame({'reviews': data,
     'label': labels
    })

output_data.to_csv('biggest_data.csv')

In [51]:
[Data_train,Data_test,Train_labels,Test_labels] = train_test_split(data,labels , test_size=0.2,stratify=labels)

In [52]:
## tokenizing the training data to find frequency of words
t = []
for line in Data_train:
    l = nltk.word_tokenize(line)
    for w in l:
        t.append(w)
        
word_features = nltk.FreqDist(t)
print (len(word_features))

55202


In [108]:
# can consider PCA(which not apply to parse matrix, so use truncated svd) and feature selection based frequency here

In [109]:
'''
for unigram
        Model  TF-IDF Accuracy
0  BernoulliNB         0.745175
1   Perceptron         0.816667
2          SVC         0.825877
3     Logistic         0.826754
for unigram & bigram
        Model  TF-IDF Accuracy
0  BernoulliNB         0.752632
1   Perceptron         0.832456
2          SVC         0.835088
3     Logistic         0.834211
采用bigram,这个数据时SVD=200
for unigram & bigram & trigram
         Model  TF-IDF Accuracy
0  BernoulliNB         0.752193
1   Perceptron         0.709211
2          SVC         0.844298
3     Logistic         0.841667
'''

'\nfor unigram\n        Model  TF-IDF Accuracy\n0  BernoulliNB         0.745175\n1   Perceptron         0.816667\n2          SVC         0.825877\n3     Logistic         0.826754\nfor unigram & bigram\n        Model  TF-IDF Accuracy\n0  BernoulliNB         0.752632\n1   Perceptron         0.832456\n2          SVC         0.835088\n3     Logistic         0.834211\n\xe9\x87\x87\xe7\x94\xa8bigram,\xe8\xbf\x99\xe4\xb8\xaa\xe6\x95\xb0\xe6\x8d\xae\xe6\x97\xb6SVD=200\nfor unigram & bigram & trigram\n         Model  TF-IDF Accuracy\n0  BernoulliNB         0.752193\n1   Perceptron         0.709211\n2          SVC         0.844298\n3     Logistic         0.841667\n'

In [53]:
# vec_all = CountVectorizer()# unigram
vec_all = CountVectorizer(ngram_range=(1,2))
# unigram & bigram
# vec_all = CountVectorizer(ngram_range=(1,3))
# unigram & bigram & trigram
ctr_features_all = vec_all.fit_transform(Data_train)

tf_vec_all = TfidfTransformer()
tr_features_all = tf_vec_all.fit_transform(ctr_features_all) # train data

cte_features_all = vec_all.transform(Data_test)
te_features_all = tf_vec_all.transform(cte_features_all) # test data

In [54]:
print tr_features_all.shape

(43311, 1005422)


In [55]:
# 不用SVD效果更差？
svd = TruncatedSVD(n_components=200)
tr_features_truncated = svd.fit_transform(tr_features_all)

te_features_truncated = svd.transform(te_features_all)

#svd = TruncatedSVD(n_components=200)
#ctr_features_truncated = svd.fit_transform(ctr_features_all)
#cte_features_truncated = svd.transform(cte_features_all)
# 此处注释掉的是不用tfidf， 只用frequency
# 2000的话在cluster上大概20-30min

In [113]:
'''
如果SVD是2000的话bigram，并无提升
 Model  TF-IDF Accuracy
0  BernoulliNB         0.735526
1   Perceptron         0.820614
2          SVC         0.843860
3     Logistic         0.837281

如果SVD=100bigram，比200降了
Model  TF-IDF Accuracy
0  BernoulliNB         0.735088
1   Perceptron         0.799561
2          SVC         0.810526
3     Logistic         0.807456

如果SVD=300，其他没啥效果，perceptron降了不少
Model  TF-IDF Accuracy
0  BernoulliNB         0.750877
1   Perceptron         0.746930
2          SVC         0.829386
3     Logistic         0.832895
('Classification report for ', 'Perceptron')
             precision    recall  f1-score   support

        neg       0.69      0.97      0.81      1247
        pos       0.92      0.48      0.63      1033

avg / total       0.80      0.75      0.73      2280
最终ensemble
neg       0.91      0.79      0.85      1441
        pos       0.71      0.87      0.78       839

avg / total       0.84      0.82      0.82      2280


还是用200吧
'''

"\n\xe5\xa6\x82\xe6\x9e\x9cSVD\xe6\x98\xaf2000\xe7\x9a\x84\xe8\xaf\x9dbigram\xef\xbc\x8c\xe5\xb9\xb6\xe6\x97\xa0\xe6\x8f\x90\xe5\x8d\x87\n Model  TF-IDF Accuracy\n0  BernoulliNB         0.735526\n1   Perceptron         0.820614\n2          SVC         0.843860\n3     Logistic         0.837281\n\n\xe5\xa6\x82\xe6\x9e\x9cSVD=100bigram\xef\xbc\x8c\xe6\xaf\x94200\xe9\x99\x8d\xe4\xba\x86\nModel  TF-IDF Accuracy\n0  BernoulliNB         0.735088\n1   Perceptron         0.799561\n2          SVC         0.810526\n3     Logistic         0.807456\n\n\xe5\xa6\x82\xe6\x9e\x9cSVD=300\xef\xbc\x8c\xe5\x85\xb6\xe4\xbb\x96\xe6\xb2\xa1\xe5\x95\xa5\xe6\x95\x88\xe6\x9e\x9c\xef\xbc\x8cperceptron\xe9\x99\x8d\xe4\xba\x86\xe4\xb8\x8d\xe5\xb0\x91\nModel  TF-IDF Accuracy\n0  BernoulliNB         0.750877\n1   Perceptron         0.746930\n2          SVC         0.829386\n3     Logistic         0.832895\n('Classification report for ', 'Perceptron')\n             precision    recall  f1-score   support\n\n        ne

In [114]:
# here searchgrid for optimal parameter， 或者直接用500的参数

In [56]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
# 'MultinomialNB': MultinomialNB(),
models = {
          'Logistic' : linear_model.LogisticRegression(C=1e5),
          'SVC' : SVC(C=10000),
        'Perceptron': linear_model.Perceptron(n_iter=1000)}

In [57]:
# 不用SVD反而差了
'''
Model  TF-IDF Accuracy
0  BernoulliNB         0.702632
1   Perceptron         0.809649
2          SVC         0.771491
3     Logistic         0.832018
'''
results_svd = pd.DataFrame()

foldnum = 0
tfprediction = {}
cprediction = {}
for name,model in models.items():
        #model.fit(tr_features_all, Train_labels)
        #tfprediction[name] = model.predict(te_features_all)
        model.fit(tr_features_truncated, Train_labels)
        tfprediction[name] = model.predict(te_features_truncated)
        tfaccuracy = metrics.accuracy_score(tfprediction[name],Test_labels)
        
        #model.fit(ctr_features_truncated,Train_labels)
        #cprediction[name] = model.predict(cte_features_truncated)
        #caccuracy = metrics.accuracy_score(cprediction[name],Test_labels)

        results_svd.loc[foldnum,'Model']=name
        results_svd.loc[foldnum,'TF-IDF Accuracy']=tfaccuracy
        #results_svd.loc[foldnum,'Count Accuracy']=caccuracy
        foldnum = foldnum+1
print (results_svd)

        Model  TF-IDF Accuracy
0  Perceptron         0.336627
1         SVC         0.954008
2    Logistic         0.955948


In [ ]:

for name,model in models.items():
    print ("Classification report for ",name)
    print(metrics.classification_report(Test_labels, tfprediction[name]))
    print("\n")


In [58]:
from sklearn.ensemble import VotingClassifier
clf2 = linear_model.LogisticRegression(C=1e5)
clf3 = SVC(C=10000)
eclf = VotingClassifier(estimators=[ ('SVC', clf3),('Logistic', clf2)], voting='hard')

In [59]:
#eclf.fit(tr_features_all, Train_labels)
eclf.fit(tr_features_truncated, Train_labels)

VotingClassifier(estimators=[('SVC', SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)), ('Logistic', LogisticRegression(C=100000.0, class_we...ty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))],
         voting='hard', weights=None)

In [60]:
#tfaccuracy = metrics.classification_report(eclf.predict(te_features_all),Test_labels)
tfaccuracy = metrics.classification_report(eclf.predict(te_features_truncated),Test_labels)
print tfaccuracy

'''
这个是svd=200, bigram的结果
precision    recall  f1-score   support

        neg       0.86      0.83      0.85      1282
        pos       0.79      0.82      0.81       998

avg / total       0.83      0.83      0.83      2280
'''

             precision    recall  f1-score   support

        neg       0.13      0.59      0.22       111
        pos       1.00      0.96      0.98     10717

avg / total       0.99      0.96      0.97     10828



'\n\xe8\xbf\x99\xe4\xb8\xaa\xe6\x98\xafsvd=200, bigram\xe7\x9a\x84\xe7\xbb\x93\xe6\x9e\x9c\nprecision    recall  f1-score   support\n\n        neg       0.86      0.83      0.85      1282\n        pos       0.79      0.82      0.81       998\n\navg / total       0.83      0.83      0.83      2280\n'

In [ ]:
# get important features
# count_vect.vocabulary_.get(u'algorithm')
# 这里小小作弊一下

In [61]:
text = 'nice amazing excellent, Like, good, love'
def find_features(text):
    text = preprocessing(text)
    important_features = []
    comments_new = list(text)
    vec_comments_new = vec_all.transform(comments_new)
    tfidf_comments_new = tf_vec_all.transform(vec_comments_new)
    
    tfidf_comments_new_truncated = svd.transform(tfidf_comments_new)
    final_decision = eclf.predict(tfidf_comments_new_truncated)[0]
    
    #final_decision = eclf.predict(tfidf_comments_new)[0]
    
    print final_decision
    text_list = text.split()
    for i in text_list:
        word = list(i)
        vec_word = vec_all.transform(word)
        
        tfidf_word = tf_vec_all.transform(vec_word)
        
        tfidf_word_truncated = svd.transform(tfidf_word)
        print eclf.predict(tfidf_word_truncated)[0]
        #print eclf.predict(tfidf_word)[0]
        
        if eclf.predict(tfidf_word_truncated)[0]==final_decision:
        #if eclf.predict(tfidf_word)[0]==final_decision:
            important_features.append(i)
    important_features.append(final_decision)
    return important_features
# so the last element of important_features would be the classification result 
find_features(text)

pos
pos
pos
pos
pos
pos
pos


['nice', 'amazing', 'excellent', 'like', 'good', 'love', 'pos']

In [62]:

from sklearn.externals import joblib
# dump to pickle
joblib.dump(eclf, 'model.pkl', compress=1)

# and reload from pickle
#eclf = joblib.load('model.pkl')


['model.pkl']